In [13]:
import toksearch
import numpy as np
import matplotlib.pyplot as plt

from toksearch import MdsSignal, Pipeline

In [ ]:
import os
import MDSplus as mds
os.environ['d3dpcs_path']='/fusion/projects/codes/pcs/data/ptdata/uncomp/mdsdata'
os.environ['d3d_path'] = 'atlas.gat.com::'

In [ ]:
shotn = 205801 

connection = mds.Connection('atlas.gat.com')
connection.openTree('d3d', shotn)

x_times = connection.get('dim_of(PTDATA(\'EOXTARGET\', 205803))')
x = connection.get('PTDATA(\'EOXTARGET\', 205803)')